In [1]:
import os
os.chdir('../')
%pwd

'/home/paladin/Downloads/Text_Summerizer/Text-Summerizer-Project'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    trained_model_path: Path
    tokenizer_path:Path
    metric_file_name: Path

In [3]:
from textSummerizer.constants import *
from textSummerizer.utils import read_yaml, create_directories

In [4]:
class configurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation       

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            data_path= self.config.data_transformation.data_path,
            trained_model_path= config.trained_model_path,
            tokenizer_path= config.tokenizer_path,
            metric_file_name = config.metric_file_name
            
        )

        return model_evaluation_config

In [5]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk, load_metric
import tensorflow as tf
import pandas as pd
from tqdm import tqdm

/home/paladin/Downloads/Text_Summerizer/Text-Summerizer-Project/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-09-12 10:29:01.571555: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-12 10:29:01.668415: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-12 10:29:01.670841: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-12 10:29:02.793842: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """
        split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements.
        """
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]



    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                               batch_size=16,
                               column_text="article",
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="tf")

            summaries = model.generate(input_ids=inputs["input_ids"],
                         attention_mask=inputs["attention_mask"],
                         length_penalty=0.8,
                         num_beams=8,
                         max_length=128)
            """
            parameter for length penalty ensures that the model does not generate sequences that are too long
            """

            # Finally, we decode the generated texts,
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True) for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score
    

    def evaluate(self):
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_t5_small = TFAutoModelForSeq2SeqLM.from_pretrained(self.config.trained_model_path)

        #loading data
        dataset_samsum = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(dataset_samsum['test'][0:10], 
                                                 rouge_metric, model_t5_small, 
                                                 tokenizer, 
                                                 batch_size = 2, 
                                                 column_text = 'dialogue', 
                                                 column_summary= 'summary'
                                                )

        rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

        df = pd.DataFrame(rouge_dict, index = [f't5-small'] )
        df.to_csv(self.config.metric_file_name, index=False)

In [7]:
import sys
from textSummerizer.exception import CustomException

In [8]:
try:
    config = configurationManager()    
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config= model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise CustomException(e, sys)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at artifacts/model_trainer/t5-small-samsum-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
/tmp/ipykernel_12139/1130015530.py:61: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
  0%|          | 0/5 [00:00<?, ?it/s]2023-09-12 10:29:12.256455: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x14093790 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-09-12 10:29:12.256501: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor devic